<a href="https://colab.research.google.com/github/ziyunxiao/Stock-Prediction-Models/blob/develop/rs_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Network

# Part 0 - Prepare packages

[Yahoo fin](http://theautomatic.net/yahoo_fin-documentation/)


In [ ]:
#! pip install yahoo_fin 

In [ ]:
!python --version

## Part 1 - Data Preprocessing

### Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importing the training set

In [ ]:
# dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
# training_set = dataset_train.iloc[:, 1:2].values
from yahoo_fin import stock_info as si
dataset_train = si.get_data('goog' , start_date = '01/01/2010', end_date= '01/01/2021')
training_set = dataset_train.iloc[:, 3:4].values

training_set


In [ ]:
dataset_train.head(5)


### Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

### Creating a data structure with 60 timesteps and 1 output

In [ ]:
X_train = []
y_train = []
for i in range(60, len(training_set)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)



In [ ]:
print(X_train.shape,y_train.shape)

### Reshaping

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_train.shape[1])

## Part 2 - Building and Training the RNN

### Importing the Keras libraries and packages

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

### Initialising the RNN

In [ ]:
regressor = Sequential()

### Adding the first LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

### Adding a second LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

### Adding a third LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

### Adding a fourth LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 100))
regressor.add(Dropout(0.2))

### Adding the output layer

In [ ]:
regressor.add(Dense(units = 1))

### Compiling the RNN

In [ ]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

### Fitting the RNN to the Training set

In [ ]:
hist = regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
# save model
# ! rm -rf google.pth/
# ! rm -rf models/

regressor.save("models/google")


In [ ]:
# load model
from tensorflow import keras
model = keras.models.load_model('models/google')

## Part 3 - Making the predictions and visualising the results

### Getting the real stock price of current year

In [ ]:
#dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
from yahoo_fin import stock_info as si
dataset_test = si.get_data('goog' , start_date = '01/01/2021')

real_stock_price = dataset_test.iloc[:, 3:4].values

In [ ]:
dataset_test.head()

### Getting the predicted stock price of 2017

In [ ]:
# dataset_train.iloc[len(dataset_train) - 60:, 3:4]['close']
# dataset_test['close']

In [ ]:
dataset_total = pd.concat((dataset_train.iloc[len(dataset_train) - 60:, 3:4]['close'], dataset_test['close']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, len(inputs)):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
# X_test

### Visualising the results

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()